In [ ]:
import torch.nn as nn
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00


**BASIC CODE**

In [ ]:
def forwardpass(text_embeds,image_embeds,logit_scale_initial=2.6529):
  logit_scale = nn.Parameter(torch.tensor(logit_scale_initial))
  image_embeds = image_embed / image_embed.norm(p=2, dim=-1, keepdim=True)
  text_embeds = text_embed / text_embed.norm(p=2, dim=-1, keepdim=True)
  logit_scale =logit_scale.exp()
  logits_per_text = torch.matmul(text_embeds, image_embeds.t()) * logit_scale
  logits_per_image = logits_per_text.t()
  return logits_per_image

In [ ]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel,AutoTokenizer,AutoProcessor

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor=AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

data_list=['cats.jpg','dog.jpg']
img_list=[]
for i in data_list:
  image = Image.open(i)
  img_list.append(image)

text_token = tokenizer(text=["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding=True)

#These are the text embedding that are generated once
text_embed=model.get_text_features(**text_token)

probs_list=[]
for i in img_list:
  image_processed = processor(images=i, return_tensors="pt", padding=True)
  image_embed=model.get_image_features(**image_processed) #Image embeddings are generated for each image at it's iteration
  output=forwardpass(text_embed,image_embed)
  probs=output.softmax(dim=1)
  probs_list.append(probs)


In [ ]:
probs_list

[tensor([[0.6958, 0.3042]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.2394, 0.7606]], grad_fn=<SoftmaxBackward0>)]